# Advanced SQL: Date/Time, Windows & Aggregations - Interactive Demo

Welcome! This demo builds on your knowledge of complex data types and teaches advanced SQL concepts.

---

## 🔗 Building on Previous Knowledge

In the **Complex Data Types** demo, you learned:
* ✅ Working with STRUCT and ARRAY
* ✅ Using EXPLODE to flatten data
* ✅ JSON operations

Now we'll add more powerful SQL techniques!

---

## 🎯 What You'll Learn

### **1. Date and Time Handling**
* Extract date parts (year, month, day, hour)
* Format dates and timestamps
* Date arithmetic and calculations
* Time zone handling

### **2. Interval Operations**
* INTERVAL syntax for date math
* Add/subtract time periods
* Calculate date differences

### **3. Window Functions**
* ROW_NUMBER, RANK, DENSE_RANK
* LAG and LEAD for time-series
* Running totals and moving averages
* Partitioning and ordering

### **4. Advanced Aggregations**
* PIVOT - Reshape data from rows to columns
* ROLLUP - Hierarchical subtotals
* CUBE - Multi-dimensional aggregations
* GROUPING SETS - Custom groupings

---

## 📚 Demo Dataset

We'll continue using **Wanderbricks** sample data:
* **bookings** - Reservation data with dates
* **customer_support_logs** - Tickets with timestamps
* **clickstream** - Events with time-series data

**Let's get started!** 🚀

## 1. Date and Time Functions 📅

**Working with dates and times is essential for:**
* Time-series analysis
* Booking and reservation systems
* Event tracking
* Trend analysis
* Reporting by time periods

**Key concepts:**
* **DATE** - Date only (YYYY-MM-DD)
* **TIMESTAMP** - Date and time with milliseconds
* **Extraction** - Get parts like year, month, day, hour
* **Formatting** - Convert to different string formats
* **Arithmetic** - Add/subtract time periods

Let's explore using the bookings table!

In [0]:
%sql
-- View the bookings table with date columns
SELECT 
  booking_id,
  user_id,
  check_in,
  check_out,
  created_at,
  updated_at
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Extract individual components from dates and timestamps

SELECT 
  booking_id,
  check_in,
  
  -- Extract from DATE
  YEAR(check_in) AS check_in_year,
  MONTH(check_in) AS check_in_month,
  DAY(check_in) AS check_in_day,
  DAYOFWEEK(check_in) AS day_of_week,  -- 1=Sunday, 7=Saturday
  DAYOFYEAR(check_in) AS day_of_year,
  QUARTER(check_in) AS quarter,
  
  -- Extract from TIMESTAMP
  created_at,
  HOUR(created_at) AS created_hour,
  MINUTE(created_at) AS created_minute,
  SECOND(created_at) AS created_second
  
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Format dates and timestamps as strings

SELECT 
  booking_id,
  check_in,
  
  -- Different date formats
  DATE_FORMAT(check_in, 'yyyy-MM-dd') AS iso_format,
  DATE_FORMAT(check_in, 'MM/dd/yyyy') AS us_format,
  DATE_FORMAT(check_in, 'dd-MMM-yyyy') AS readable_format,
  DATE_FORMAT(check_in, 'EEEE, MMMM dd, yyyy') AS full_format,
  
  -- Timestamp formatting
  DATE_FORMAT(created_at, 'yyyy-MM-dd HH:mm:ss') AS datetime_format
  
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Truncate dates to specific time units
-- Useful for grouping by time periods

SELECT 
  booking_id,
  created_at,
  
  -- Truncate to different levels
  DATE_TRUNC('YEAR', created_at) AS year_start,
  DATE_TRUNC('QUARTER', created_at) AS quarter_start,
  DATE_TRUNC('MONTH', created_at) AS month_start,
  DATE_TRUNC('WEEK', created_at) AS week_start,
  DATE_TRUNC('DAY', created_at) AS day_start,
  DATE_TRUNC('HOUR', created_at) AS hour_start
  
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Get current date and time values

SELECT 
  CURRENT_DATE() AS today,
  CURRENT_TIMESTAMP() AS now,
  CURRENT_TIMEZONE() AS timezone,
  
  -- Unix timestamp (seconds since 1970-01-01)
  UNIX_TIMESTAMP() AS unix_now,
  UNIX_TIMESTAMP(CURRENT_TIMESTAMP()) AS unix_now_explicit

In [0]:
%sql
-- Add and subtract days from dates

SELECT 
  booking_id,
  check_in,
  check_out,
  
  -- Add/subtract days
  DATE_ADD(check_in, 7) AS one_week_later,
  DATE_SUB(check_in, 7) AS one_week_earlier,
  
  -- Calculate differences
  DATEDIFF(check_out, check_in) AS nights_stayed,
  DATEDIFF(CURRENT_DATE(), check_in) AS days_since_checkin
  
FROM samples.wanderbricks.bookings
LIMIT 10

### 📚 Date/Time Functions Reference

**Extraction:**
```sql
YEAR(date), MONTH(date), DAY(date)
HOUR(timestamp), MINUTE(timestamp), SECOND(timestamp)
DAYOFWEEK(date), DAYOFYEAR(date), QUARTER(date)
WEEKOFYEAR(date)
```

**Formatting:**
```sql
DATE_FORMAT(date, 'yyyy-MM-dd')
DATE_FORMAT(timestamp, 'yyyy-MM-dd HH:mm:ss')
```

**Truncation:**
```sql
DATE_TRUNC('YEAR|QUARTER|MONTH|WEEK|DAY|HOUR', timestamp)
```

**Arithmetic:**
```sql
DATE_ADD(date, days)
DATE_SUB(date, days)
DATEDIFF(date1, date2)  -- Returns days between
MONTHS_BETWEEN(date1, date2)  -- Returns months between
```

**Current Values:**
```sql
CURRENT_DATE(), CURRENT_TIMESTAMP()
NOW()  -- Alias for CURRENT_TIMESTAMP()
```

**Conversion:**
```sql
TO_DATE('2024-01-15')  -- String to date
TO_TIMESTAMP('2024-01-15 10:30:00')  -- String to timestamp
CAST(string_col AS DATE)
```

## 2. Interval Operations ⏱️

**What is an INTERVAL?**

INTERVAL represents a duration of time that can be added or subtracted from dates/timestamps.

**Syntax:**
```sql
INTERVAL n UNIT
```

**Supported units:**
* YEAR, MONTH, WEEK, DAY
* HOUR, MINUTE, SECOND
* Combinations: `INTERVAL '1-6' YEAR TO MONTH`

**Why use INTERVAL?**
* More readable than DATE_ADD/DATE_SUB
* Supports complex time periods
* Standard SQL syntax
* Works with both DATE and TIMESTAMP

In [0]:
%sql
-- Add and subtract intervals from dates

SELECT 
  booking_id,
  check_in,
  
  -- Add intervals
  check_in + INTERVAL 1 DAY AS tomorrow,
  check_in + INTERVAL 1 WEEK AS next_week,
  check_in + INTERVAL 1 MONTH AS next_month,
  check_in + INTERVAL 1 YEAR AS next_year,
  
  -- Subtract intervals
  check_in - INTERVAL 7 DAYS AS week_ago
  
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Work with time intervals (hours, minutes, seconds)

SELECT 
  booking_id,
  created_at,
  
  -- Add time intervals
  created_at + INTERVAL 2 HOURS AS two_hours_later,
  created_at + INTERVAL 30 MINUTES AS thirty_min_later,
  created_at + INTERVAL 90 SECONDS AS ninety_sec_later,
  
  -- Combine date and time intervals
  created_at + INTERVAL 1 DAY + INTERVAL 2 HOURS AS tomorrow_plus_2hrs
  
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Use complex interval expressions

SELECT 
  booking_id,
  check_in,
  check_out,
  
  -- Multiple units
  check_in + INTERVAL '1-6' YEAR TO MONTH AS year_and_half_later,
  check_in + INTERVAL '2 10:30:00' DAY TO SECOND AS complex_interval,
  
  -- Calculate booking window
  check_in - INTERVAL 30 DAYS AS booking_window_start,
  check_in - INTERVAL 1 DAY AS day_before_checkin
  
FROM samples.wanderbricks.bookings
LIMIT 10

In [0]:
%sql
-- Real-world date calculations for business logic

SELECT 
  booking_id,
  check_in,
  check_out,
  created_at,
  
  -- Calculate stay duration
  DATEDIFF(check_out, check_in) AS nights,
  
  -- Calculate booking lead time (how far in advance)
  DATEDIFF(check_in, CAST(created_at AS DATE)) AS lead_time_days,
  
  -- Check if booking is in the future
  CASE 
    WHEN check_in > CURRENT_DATE() THEN 'Future'
    WHEN check_in = CURRENT_DATE() THEN 'Today'
    ELSE 'Past'
  END AS booking_status,
  
  -- Calculate cancellation deadline (24 hours before check-in)
  check_in - INTERVAL 1 DAY AS cancellation_deadline
  
FROM samples.wanderbricks.bookings
LIMIT 10

### ✅ Interval Best Practices

**Use INTERVAL for readability:**
```sql
-- Good: Clear and readable
check_in + INTERVAL 7 DAYS

-- Also works: But less clear
DATE_ADD(check_in, 7)
```

**Combine multiple intervals:**
```sql
-- Add multiple time periods
date_col + INTERVAL 1 MONTH + INTERVAL 2 DAYS + INTERVAL 3 HOURS
```

**Use for filtering:**
```sql
-- Find bookings in the last 30 days
WHERE created_at >= CURRENT_DATE() - INTERVAL 30 DAYS

-- Find upcoming bookings
WHERE check_in BETWEEN CURRENT_DATE() AND CURRENT_DATE() + INTERVAL 7 DAYS
```

**Common patterns:**
```sql
-- Last 7 days
WHERE date_col >= CURRENT_DATE() - INTERVAL 7 DAYS

-- Next 30 days
WHERE date_col BETWEEN CURRENT_DATE() AND CURRENT_DATE() + INTERVAL 30 DAYS

-- Same day last year
WHERE date_col = CURRENT_DATE() - INTERVAL 1 YEAR
```

## 3. Window Functions 🔲

**What are Window Functions?**

Window functions perform calculations across a set of rows related to the current row, **without collapsing rows** like GROUP BY.

**Key difference from GROUP BY:**
* **GROUP BY** - Collapses rows into groups
* **Window Functions** - Keep all rows, add calculated columns

**Common use cases:**
* Ranking (top N per category)
* Running totals and moving averages
* Comparing with previous/next rows
* Percentiles and distributions
* Time-series analysis

**Syntax:**
```sql
function() OVER (
  PARTITION BY column  -- Optional: group by
  ORDER BY column      -- Optional: sort within partition
  ROWS/RANGE ...       -- Optional: frame specification
)
```

In [0]:
%sql
-- ROW_NUMBER assigns a unique sequential number to each row
-- Useful for ranking and pagination

SELECT 
  booking_id,
  user_id,
  check_in,
  total_amount,
  
  -- Number all bookings sequentially
  ROW_NUMBER() OVER (ORDER BY check_in) AS booking_sequence,
  
  -- Number bookings per user
  ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY check_in) AS user_booking_number
  
FROM samples.wanderbricks.bookings
LIMIT 20

In [0]:
%sql
-- RANK and DENSE_RANK for handling ties

SELECT 
  booking_id,
  user_id,
  total_amount,
  
  -- ROW_NUMBER: Always unique (1, 2, 3, 4, 5...)
  ROW_NUMBER() OVER (ORDER BY total_amount DESC) AS row_num,
  
  -- RANK: Gaps after ties (1, 2, 2, 4, 5...)
  RANK() OVER (ORDER BY total_amount DESC) AS rank,
  
  -- DENSE_RANK: No gaps (1, 2, 2, 3, 4...)
  DENSE_RANK() OVER (ORDER BY total_amount DESC) AS dense_rank
  
FROM samples.wanderbricks.bookings
LIMIT 20

In [0]:
%sql
-- Find top 3 bookings per user by amount
-- This is a very common pattern!

WITH ranked_bookings AS (
  SELECT 
    booking_id,
    user_id,
    check_in,
    total_amount,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY total_amount DESC) AS rank
  FROM samples.wanderbricks.bookings
)
SELECT 
  user_id,
  booking_id,
  check_in,
  total_amount,
  rank
FROM ranked_bookings
WHERE rank <= 3
ORDER BY user_id, rank
LIMIT 30

In [0]:
%sql
-- LAG and LEAD access previous/next rows
-- Perfect for time-series comparisons

SELECT 
  booking_id,
  user_id,
  check_in,
  total_amount,
  
  -- LAG: Get previous row value
  LAG(total_amount) OVER (PARTITION BY user_id ORDER BY check_in) AS previous_booking_amount,
  LAG(check_in) OVER (PARTITION BY user_id ORDER BY check_in) AS previous_check_in,
  
  -- LEAD: Get next row value
  LEAD(total_amount) OVER (PARTITION BY user_id ORDER BY check_in) AS next_booking_amount,
  
  -- Calculate change from previous booking
  total_amount - LAG(total_amount) OVER (PARTITION BY user_id ORDER BY check_in) AS amount_change
  
FROM samples.wanderbricks.bookings
LIMIT 30

In [0]:
%sql
-- Calculate running totals (cumulative sums)
-- Shows cumulative revenue per user over time

SELECT 
  booking_id,
  user_id,
  check_in,
  total_amount,
  
  -- Running total per user
  SUM(total_amount) OVER (
    PARTITION BY user_id 
    ORDER BY check_in
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS cumulative_spent,
  
  -- Count of bookings so far
  COUNT(*) OVER (
    PARTITION BY user_id 
    ORDER BY check_in
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS booking_count
  
FROM samples.wanderbricks.bookings
WHERE user_id IN (SELECT user_id FROM samples.wanderbricks.bookings LIMIT 5)
ORDER BY user_id, check_in
LIMIT 30

In [0]:
%sql
-- Calculate moving averages
-- Shows average of current row and previous 2 rows (3-row window)

SELECT 
  DATE_TRUNC('DAY', created_at) AS booking_date,
  COUNT(*) AS daily_bookings,
  SUM(total_amount) AS daily_revenue,
  
  -- 3-day moving average of bookings
  AVG(COUNT(*)) OVER (
    ORDER BY DATE_TRUNC('DAY', created_at)
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ) AS moving_avg_bookings,
  
  -- 3-day moving average of revenue
  ROUND(AVG(SUM(total_amount)) OVER (
    ORDER BY DATE_TRUNC('DAY', created_at)
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
  ), 2) AS moving_avg_revenue
  
FROM samples.wanderbricks.bookings
GROUP BY DATE_TRUNC('DAY', created_at)
ORDER BY booking_date
LIMIT 30

### 📊 Window Frame Specifications

**Frame types:**

**ROWS** - Physical rows
```sql
ROWS BETWEEN 2 PRECEDING AND CURRENT ROW  -- Last 3 rows
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW  -- All rows up to current
ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING  -- Current + next 2
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING  -- All rows
```

**RANGE** - Logical range based on values
```sql
RANGE BETWEEN INTERVAL 7 DAYS PRECEDING AND CURRENT ROW  -- Last 7 days
RANGE BETWEEN 100 PRECEDING AND 100 FOLLOWING  -- Values within ±100
```

**Common patterns:**
```sql
-- Running total
SUM(amount) OVER (ORDER BY date ROWS UNBOUNDED PRECEDING)

-- Moving average (last 7 rows)
AVG(value) OVER (ORDER BY date ROWS 6 PRECEDING)

-- Compare with previous row
LAG(value, 1) OVER (ORDER BY date)

-- Rank within groups
RANK() OVER (PARTITION BY category ORDER BY value DESC)
```

### 📚 Window Functions Reference

**Ranking Functions:**
* `ROW_NUMBER()` - Unique sequential number
* `RANK()` - Rank with gaps for ties
* `DENSE_RANK()` - Rank without gaps
* `NTILE(n)` - Divide into n buckets
* `PERCENT_RANK()` - Relative rank (0 to 1)

**Value Functions:**
* `LAG(col, offset)` - Previous row value
* `LEAD(col, offset)` - Next row value
* `FIRST_VALUE(col)` - First value in window
* `LAST_VALUE(col)` - Last value in window
* `NTH_VALUE(col, n)` - Nth value in window

**Aggregate Functions (as window functions):**
* `SUM(col) OVER (...)` - Running sum
* `AVG(col) OVER (...)` - Moving average
* `COUNT(*) OVER (...)` - Running count
* `MIN(col) OVER (...)` - Running minimum
* `MAX(col) OVER (...)` - Running maximum

## 4. PIVOT - Reshape Data 🔄

**What is PIVOT?**

PIVOT transforms rows into columns, creating a cross-tabulation (crosstab) view of your data.

**Before PIVOT:**
```
month  | status    | count
-------|-----------|------
Jan    | confirmed | 100
Jan    | cancelled | 10
Feb    | confirmed | 120
Feb    | cancelled | 15
```

**After PIVOT:**
```
month  | confirmed | cancelled
-------|-----------|----------
Jan    | 100       | 10
Feb    | 120       | 15
```

**Common use cases:**
* Monthly/quarterly reports
* Cross-tabulation analysis
* Comparison matrices
* Dashboard-ready data
* Excel-style pivot tables

In [0]:
%sql
-- PIVOT booking counts by month and status
-- Transform status values into columns

SELECT *
FROM (
  SELECT 
    DATE_FORMAT(check_in, 'yyyy-MM') AS booking_month,
    status,
    booking_id
  FROM samples.wanderbricks.bookings
)
PIVOT (
  COUNT(booking_id)
  FOR status IN ('confirmed', 'cancelled', 'pending')
)
ORDER BY booking_month
LIMIT 20

In [0]:
%sql
-- PIVOT with different aggregation functions
-- Show total revenue by month and status

SELECT *
FROM (
  SELECT 
    DATE_FORMAT(check_in, 'yyyy-MM') AS booking_month,
    status,
    total_amount
  FROM samples.wanderbricks.bookings
)
PIVOT (
  ROUND(SUM(total_amount), 2) AS total_revenue
  FOR status IN ('confirmed', 'cancelled', 'pending')
)
ORDER BY booking_month
LIMIT 20

In [0]:
%sql
-- PIVOT with multiple aggregation functions
-- Show both count and sum for each status

SELECT *
FROM (
  SELECT 
    DATE_FORMAT(check_in, 'yyyy-MM') AS booking_month,
    status,
    total_amount
  FROM samples.wanderbricks.bookings
)
PIVOT (
  COUNT(*) AS count,
  ROUND(SUM(total_amount), 2) AS revenue
  FOR status IN ('confirmed', 'cancelled', 'pending')
)
ORDER BY booking_month
LIMIT 20

In [0]:
%sql
-- PIVOT by day of week
-- Show booking patterns across weekdays

SELECT *
FROM (
  SELECT 
    YEAR(check_in) AS year,
    MONTH(check_in) AS month,
    DAYOFWEEK(check_in) AS day_of_week,
    booking_id
  FROM samples.wanderbricks.bookings
)
PIVOT (
  COUNT(booking_id)
  FOR day_of_week IN (1, 2, 3, 4, 5, 6, 7)
)
ORDER BY year, month
LIMIT 20

### ✅ PIVOT Best Practices

**Basic syntax:**
```sql
SELECT *
FROM (
  SELECT grouping_col, pivot_col, value_col
  FROM table
)
PIVOT (
  AGG_FUNCTION(value_col)
  FOR pivot_col IN (value1, value2, value3)
)
```

**Tips:**

✅ **Specify pivot values explicitly** - List all expected values in IN clause  
✅ **Use meaningful aliases** - Name aggregations clearly  
✅ **Limit pivot columns** - Too many columns make results hard to read  
✅ **Pre-aggregate if needed** - PIVOT works best on pre-aggregated data  

**Common mistakes:**

❌ Don't pivot on high-cardinality columns (creates too many columns)  
❌ Don't forget to specify aggregation function  
❌ Don't pivot without understanding the data distribution  

**Alternative to PIVOT:**
```sql
-- Use CASE WHEN for more control
SELECT 
  month,
  SUM(CASE WHEN status = 'confirmed' THEN 1 ELSE 0 END) AS confirmed,
  SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) AS cancelled
FROM table
GROUP BY month
```

## 5. Advanced Aggregations 📈

**Beyond simple GROUP BY:**

* **ROLLUP** - Hierarchical subtotals (grand total + subtotals)
* **CUBE** - All possible combinations of groupings
* **GROUPING SETS** - Custom grouping combinations

**Why use them?**
* Generate subtotals and grand totals in one query
* Create multi-dimensional reports
* Avoid multiple queries with UNION
* Perfect for business reporting

Let's explore each one!

In [0]:
%sql
-- First, let's see a regular GROUP BY for comparison
-- Bookings by year and month

SELECT 
  YEAR(check_in) AS year,
  MONTH(check_in) AS month,
  COUNT(*) AS booking_count,
  ROUND(SUM(total_amount), 2) AS total_revenue
FROM samples.wanderbricks.bookings
GROUP BY YEAR(check_in), MONTH(check_in)
ORDER BY year, month
LIMIT 20

In [0]:
%sql
-- ROLLUP creates hierarchical subtotals
-- Generates: (year, month), (year), and grand total

SELECT 
  YEAR(check_in) AS year,
  MONTH(check_in) AS month,
  COUNT(*) AS booking_count,
  ROUND(SUM(total_amount), 2) AS total_revenue
FROM samples.wanderbricks.bookings
GROUP BY ROLLUP(YEAR(check_in), MONTH(check_in))
ORDER BY year NULLS LAST, month NULLS LAST
LIMIT 30

### 🔍 Understanding ROLLUP Output

**ROLLUP generates these groupings:**

1. **(year, month)** - Detailed level
2. **(year, NULL)** - Year subtotals
3. **(NULL, NULL)** - Grand total

**How to identify subtotal rows:**
* NULL values indicate aggregation level
* Use GROUPING() function to detect

**Example output:**
```
year | month | count | revenue
-----|-------|-------|--------
2024 | 1     | 100   | 50000   ← Jan 2024
2024 | 2     | 120   | 60000   ← Feb 2024
2024 | NULL  | 220   | 110000  ← 2024 Total
2025 | 1     | 150   | 75000   ← Jan 2025
2025 | NULL  | 150   | 75000   ← 2025 Total
NULL | NULL  | 370   | 185000  ← Grand Total
```

In [0]:
%sql
-- Use GROUPING() to identify subtotal rows
-- GROUPING() returns 1 for aggregated (NULL) columns, 0 otherwise

SELECT 
  YEAR(check_in) AS year,
  MONTH(check_in) AS month,
  COUNT(*) AS booking_count,
  ROUND(SUM(total_amount), 2) AS total_revenue,
  
  -- Identify aggregation level
  GROUPING(YEAR(check_in)) AS is_year_total,
  GROUPING(MONTH(check_in)) AS is_month_total,
  
  -- Create labels for subtotal rows
  CASE 
    WHEN GROUPING(YEAR(check_in)) = 1 THEN 'GRAND TOTAL'
    WHEN GROUPING(MONTH(check_in)) = 1 THEN 'YEAR TOTAL'
    ELSE 'DETAIL'
  END AS row_type
  
FROM samples.wanderbricks.bookings
GROUP BY ROLLUP(YEAR(check_in), MONTH(check_in))
ORDER BY year NULLS LAST, month NULLS LAST
LIMIT 30

In [0]:
%sql
-- CUBE generates all possible combinations of groupings
-- For 2 columns: (col1, col2), (col1), (col2), (grand total)

SELECT 
  YEAR(check_in) AS year,
  status,
  COUNT(*) AS booking_count,
  ROUND(SUM(total_amount), 2) AS total_revenue,
  
  -- Label the aggregation level
  CASE 
    WHEN GROUPING(YEAR(check_in)) = 1 AND GROUPING(status) = 1 THEN 'GRAND TOTAL'
    WHEN GROUPING(YEAR(check_in)) = 1 THEN 'STATUS TOTAL'
    WHEN GROUPING(status) = 1 THEN 'YEAR TOTAL'
    ELSE 'DETAIL'
  END AS aggregation_level
  
FROM samples.wanderbricks.bookings
GROUP BY CUBE(YEAR(check_in), status)
ORDER BY year NULLS LAST, status NULLS LAST
LIMIT 40

In [0]:
%sql
-- GROUPING SETS lets you specify exactly which groupings you want
-- More flexible than ROLLUP or CUBE

SELECT 
  YEAR(check_in) AS year,
  MONTH(check_in) AS month,
  status,
  COUNT(*) AS booking_count,
  ROUND(SUM(total_amount), 2) AS total_revenue
FROM samples.wanderbricks.bookings
GROUP BY GROUPING SETS (
  (YEAR(check_in), MONTH(check_in), status),  -- Full detail
  (YEAR(check_in), status),                    -- By year and status
  (status),                                     -- By status only
  ()                                            -- Grand total
)
ORDER BY year NULLS LAST, month NULLS LAST, status NULLS LAST
LIMIT 40

### 🔄 ROLLUP vs CUBE vs GROUPING SETS

**For columns (A, B, C):**

**ROLLUP(A, B, C)** - Hierarchical (4 groupings):
```
(A, B, C)  ← Most detailed
(A, B)
(A)
()         ← Grand total
```

**CUBE(A, B, C)** - All combinations (8 groupings):
```
(A, B, C)
(A, B)
(A, C)
(B, C)
(A)
(B)
(C)
()         ← Grand total
```

**GROUPING SETS** - Custom (you choose):
```sql
GROUPING SETS (
  (A, B),
  (A),
  ()
)
```

**When to use:**

* **ROLLUP** - Hierarchical reports (Year → Quarter → Month)
* **CUBE** - Multi-dimensional analysis (all combinations)
* **GROUPING SETS** - Specific groupings you need (most efficient)

## 6. Real-World Patterns 🎯

Let's combine everything we've learned to solve real business problems!

In [0]:
%sql
-- Analyze booking trends over time with window functions
-- Compare each day with previous day and calculate running totals

WITH daily_bookings AS (
  SELECT 
    DATE_TRUNC('DAY', created_at) AS booking_date,
    COUNT(*) AS bookings,
    ROUND(SUM(total_amount), 2) AS revenue
  FROM samples.wanderbricks.bookings
  GROUP BY DATE_TRUNC('DAY', created_at)
)
SELECT 
  booking_date,
  bookings,
  revenue,
  
  -- Compare with previous day
  LAG(bookings) OVER (ORDER BY booking_date) AS prev_day_bookings,
  bookings - LAG(bookings) OVER (ORDER BY booking_date) AS booking_change,
  
  -- Running total
  SUM(bookings) OVER (ORDER BY booking_date ROWS UNBOUNDED PRECEDING) AS cumulative_bookings,
  
  -- 7-day moving average
  ROUND(AVG(bookings) OVER (
    ORDER BY booking_date 
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ), 2) AS moving_avg_7day
  
FROM daily_bookings
ORDER BY booking_date
LIMIT 30

In [0]:
%sql
-- Cohort analysis: Track user behavior over time
-- Find each user's first booking and track subsequent bookings

WITH user_bookings AS (
  SELECT 
    user_id,
    booking_id,
    check_in,
    total_amount,
    FIRST_VALUE(check_in) OVER (PARTITION BY user_id ORDER BY check_in) AS first_booking_date,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY check_in) AS booking_number
  FROM samples.wanderbricks.bookings
)
SELECT 
  user_id,
  booking_number,
  check_in,
  first_booking_date,
  DATEDIFF(check_in, first_booking_date) AS days_since_first_booking,
  total_amount
FROM user_bookings
WHERE booking_number <= 5  -- First 5 bookings per user
ORDER BY user_id, booking_number
LIMIT 30

In [0]:
%sql
-- Seasonal analysis: Compare bookings across months and years
-- PIVOT to create year-over-year comparison

SELECT *
FROM (
  SELECT 
    MONTH(check_in) AS month,
    YEAR(check_in) AS year,
    booking_id
  FROM samples.wanderbricks.bookings
)
PIVOT (
  COUNT(booking_id) AS bookings
  FOR year IN (2023, 2024, 2025)
)
ORDER BY month

In [0]:
%sql
-- Analyze support response times using timestamps from messages
-- Combine arrays, timestamps, and window functions

WITH message_times AS (
  SELECT 
    ticket_id,
    user_id,
    created_at AS ticket_created,
    messages[0].timestamp AS first_message_time,
    messages[1].timestamp AS first_response_time,
    messages[0].sender AS first_sender,
    messages[1].sender AS first_responder
  FROM samples.wanderbricks.customer_support_logs
  WHERE SIZE(messages) >= 2
)
SELECT 
  ticket_id,
  user_id,
  first_message_time,
  first_response_time,
  
  -- Calculate response time (if both are timestamps)
  -- Note: These are stored as strings in this table, so we'll show the pattern
  first_sender,
  first_responder,
  
  -- Rank tickets by response time
  ROW_NUMBER() OVER (ORDER BY first_response_time) AS response_rank
  
FROM message_times
LIMIT 20

In [0]:
%sql
-- Executive summary report with hierarchical totals
-- Year > Quarter > Month breakdown with subtotals

SELECT 
  YEAR(check_in) AS year,
  QUARTER(check_in) AS quarter,
  MONTH(check_in) AS month,
  COUNT(*) AS bookings,
  ROUND(SUM(total_amount), 2) AS revenue,
  ROUND(AVG(total_amount), 2) AS avg_booking_value,
  
  -- Label the aggregation level
  CASE 
    WHEN GROUPING(YEAR(check_in)) = 1 THEN 'GRAND TOTAL'
    WHEN GROUPING(QUARTER(check_in)) = 1 THEN CONCAT('YEAR ', CAST(YEAR(check_in) AS STRING), ' TOTAL')
    WHEN GROUPING(MONTH(check_in)) = 1 THEN CONCAT('Q', CAST(QUARTER(check_in) AS STRING), ' TOTAL')
    ELSE 'MONTHLY DETAIL'
  END AS report_level
  
FROM samples.wanderbricks.bookings
GROUP BY ROLLUP(YEAR(check_in), QUARTER(check_in), MONTH(check_in))
ORDER BY year NULLS LAST, quarter NULLS LAST, month NULLS LAST
LIMIT 50

In [0]:
%sql
-- Advanced pattern: Combine dates, windows, and PIVOT
-- Monthly booking status distribution with month-over-month change

WITH monthly_status AS (
  SELECT 
    DATE_TRUNC('MONTH', check_in) AS month,
    status,
    COUNT(*) AS booking_count
  FROM samples.wanderbricks.bookings
  GROUP BY DATE_TRUNC('MONTH', check_in), status
),
pivoted AS (
  SELECT *
  FROM monthly_status
  PIVOT (
    SUM(booking_count)
    FOR status IN ('confirmed', 'cancelled', 'pending')
  )
)
SELECT 
  month,
  confirmed,
  cancelled,
  pending,
  
  -- Month-over-month change
  confirmed - LAG(confirmed) OVER (ORDER BY month) AS confirmed_change,
  
  -- Cancellation rate
  ROUND(cancelled * 100.0 / NULLIF(confirmed + cancelled + pending, 0), 2) AS cancellation_rate
  
FROM pivoted
ORDER BY month
LIMIT 20

## 📚 Quick Reference Guide

### **Date/Time Functions**
```sql
-- Extraction
YEAR(date), MONTH(date), DAY(date), HOUR(timestamp)
DAYOFWEEK(date), QUARTER(date), WEEKOFYEAR(date)

-- Formatting
DATE_FORMAT(date, 'yyyy-MM-dd')

-- Truncation
DATE_TRUNC('MONTH', timestamp)

-- Arithmetic
DATE_ADD(date, days), DATE_SUB(date, days)
DATEDIFF(date1, date2)

-- Current values
CURRENT_DATE(), CURRENT_TIMESTAMP()
```

### **Intervals**
```sql
-- Basic intervals
date_col + INTERVAL 7 DAYS
date_col - INTERVAL 1 MONTH
timestamp_col + INTERVAL 2 HOURS

-- Complex intervals
date_col + INTERVAL '1-6' YEAR TO MONTH
date_col + INTERVAL '2 10:30:00' DAY TO SECOND

-- Filtering with intervals
WHERE date_col >= CURRENT_DATE() - INTERVAL 30 DAYS
```

### **Window Functions**
```sql
-- Ranking
ROW_NUMBER() OVER (PARTITION BY col ORDER BY col2)
RANK() OVER (ORDER BY col)
DENSE_RANK() OVER (ORDER BY col)

-- Value access
LAG(col, 1) OVER (ORDER BY date)
LEAD(col, 1) OVER (ORDER BY date)
FIRST_VALUE(col) OVER (PARTITION BY group ORDER BY date)

-- Running aggregations
SUM(col) OVER (ORDER BY date ROWS UNBOUNDED PRECEDING)
AVG(col) OVER (ORDER BY date ROWS 6 PRECEDING)
```

### **PIVOT**
```sql
SELECT *
FROM (
  SELECT group_col, pivot_col, value_col
  FROM table
)
PIVOT (
  AGG_FUNCTION(value_col)
  FOR pivot_col IN (val1, val2, val3)
)
```

### **Advanced Aggregations**
```sql
-- ROLLUP (hierarchical)
GROUP BY ROLLUP(col1, col2, col3)

-- CUBE (all combinations)
GROUP BY CUBE(col1, col2)

-- GROUPING SETS (custom)
GROUP BY GROUPING SETS ((col1, col2), (col1), ())

-- Identify subtotals
GROUPING(col) -- Returns 1 if aggregated, 0 otherwise
```

## 📊 Common Use Cases Summary

### **Time-Series Analysis**
```sql
-- Daily trends with moving averages
SELECT 
  date,
  metric,
  AVG(metric) OVER (ORDER BY date ROWS 6 PRECEDING) AS moving_avg_7day
FROM daily_metrics
```

### **Ranking and Top N**
```sql
-- Top 10 per category
WITH ranked AS (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY category ORDER BY value DESC) AS rank
  FROM table
)
SELECT * FROM ranked WHERE rank <= 10
```

### **Year-over-Year Comparison**
```sql
-- Compare with same period last year
SELECT 
  date,
  revenue,
  LAG(revenue, 365) OVER (ORDER BY date) AS revenue_last_year,
  revenue - LAG(revenue, 365) OVER (ORDER BY date) AS yoy_change
FROM daily_revenue
```

### **Hierarchical Reports**
```sql
-- Multi-level subtotals
SELECT 
  region,
  country,
  city,
  SUM(sales) AS total_sales
FROM sales_data
GROUP BY ROLLUP(region, country, city)
```

### **Cross-Tabulation**
```sql
-- Pivot for dashboard
SELECT * FROM (
  SELECT month, category, amount
  FROM sales
)
PIVOT (
  SUM(amount) FOR category IN ('A', 'B', 'C')
)
```

## ✅ Best Practices Summary

### **Date/Time Handling**

✅ Use DATE_TRUNC for grouping by time periods  
✅ Use INTERVAL for readable date arithmetic  
✅ Always consider time zones for global data  
✅ Use DATEDIFF for calculating durations  
❌ Don't compare dates as strings  

### **Window Functions**

✅ Use PARTITION BY to group calculations  
✅ Always specify ORDER BY for meaningful results  
✅ Use appropriate frame specification (ROWS vs RANGE)  
✅ Test with small datasets first (window functions can be expensive)  
❌ Don't use window functions when GROUP BY is sufficient  

### **PIVOT**

✅ Specify all expected pivot values explicitly  
✅ Use for reports and dashboards  
✅ Limit number of pivot columns (< 20)  
❌ Don't pivot on high-cardinality columns  
❌ Don't use PIVOT when CASE WHEN is clearer  

### **Advanced Aggregations**

✅ Use ROLLUP for hierarchical reports  
✅ Use CUBE for multi-dimensional analysis  
✅ Use GROUPING SETS for specific combinations  
✅ Use GROUPING() to identify subtotal rows  
❌ Don't use CUBE with many columns (exponential growth)  

### **Performance Tips**

✅ Filter data before window functions  
✅ Use appropriate partitioning  
✅ Consider materialized views for complex calculations  
✅ Test queries with EXPLAIN  
✅ Use date partitioning when available  

## 🎉 Congratulations!

You've completed the Advanced SQL demo!

### **What You Learned:**

✅ **Date/Time Functions** - Extract, format, and calculate with dates  
✅ **Interval Operations** - Readable date arithmetic with INTERVAL  
✅ **Window Functions** - ROW_NUMBER, RANK, LAG, LEAD, running totals  
✅ **PIVOT** - Reshape data from rows to columns  
✅ **Advanced Aggregations** - ROLLUP, CUBE, GROUPING SETS  
✅ **Real-World Patterns** - Combining concepts for business solutions  

---

### **Key Takeaways:**

1. **Window functions are powerful** - Calculations without collapsing rows
2. **PIVOT simplifies reporting** - Transform data for dashboards
3. **ROLLUP saves queries** - Subtotals in one query
4. **Intervals are readable** - Better than date arithmetic functions
5. **Combine techniques** - Real power comes from using them together

---

### **Building on Complex Data Types:**

You now know how to:
* Work with STRUCT and ARRAY (previous demo)
* Handle dates and time-series data (this demo)
* Perform advanced analytics (this demo)
* Create production-ready reports (this demo)

---

### **Next Steps:**

* Practice with your own time-series data
* Build executive dashboards with PIVOT and ROLLUP
* Explore CTEs and subqueries
* Learn about performance optimization
* Study Delta Lake time travel

---

### **Resources:**

* [Databricks SQL Functions](https://docs.databricks.com/sql/language-manual/sql-ref-functions.html)
* [Window Functions Guide](https://docs.databricks.com/sql/language-manual/sql-ref-syntax-qry-select-window.html)
* [Date/Time Functions](https://docs.databricks.com/sql/language-manual/sql-ref-functions-builtin.html#date-and-timestamp-functions)

---

**You're now ready for advanced SQL analytics!** 🚀

*Happy querying!*